# Create Slice for Fabric Rack
This is the slice creation for the ```TCP Traffic``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                         
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,7a5adb91-c4c0-4a1c-8021-7b6c56af196f
Bastion Username,lrco247_0000050326
Bastion Private Key File,/home/fabric/work/fabric_config/testingNEWkey
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
slice_name = "TCP_Traffic_intro"
site = 'MASS'#fablib.get_random_site()
print(f'site = {site}')

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 4
ram = 16
disk = 50

site = MASS


In [3]:
try:
    # Create a slice
    slice = fablib.new_slice(name=slice_name)
    #first Node
    ND1 = slice.add_node(name="ND_A", site=site)
    ND1.set_capacities(cores=cores, ram=ram, disk=disk)
    ND1.set_image(image)
    Net_ND1 = ND1.add_component(model=nicmodel, name="ND_A").get_interfaces()[0]
    

    #Second Node
    ND2 = slice.add_node(name="ND_B", site=site)
    ND2.set_capacities(cores=cores, ram=ram, disk=disk)
    ND2.set_image(image)
    Net_ND2 = ND2.add_component(model=nicmodel, name="ND_B").get_interfaces()[0] 

    #Lan
    LN = slice.add_l2network(name="Lan", interfaces=[Net_ND1, Net_ND2])
    # Submit the slice
    
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


Retry: 15, Time: 333 sec


ID,3412ab44-75bd-4806-8790-106c7d6587a2
Name,TCP_Traffic_intro
Lease Expiration (UTC),2023-05-29 14:22:04 +0000
Lease Start (UTC),2023-05-28 14:22:05 +0000
Project ID,7a5adb91-c4c0-4a1c-8021-7b6c56af196f
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
ac2ae1c9-4c14-441b-b1b1-eb3dd2b6205b,ND_A,4,16,100,default_ubuntu_20,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.95,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.95,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e1cc59f2-ceb0-42b9-98d1-659e0051d7c6,ND_B,4,16,100,default_ubuntu_20,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.107,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.107,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
6cfc1e06-da74-4b75-805a-5a4c8962c59b,Lan,L2,L2Bridge,MASS,None,None,Active,



Time to stable 333 seconds
Running post_boot_config ... Time to post boot config 341 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
ND_A-ND_A-p1,ND_A,Lan,100,None,06:66:35:F6:23:F7,ens7,ens7
ND_B-ND_B-p1,ND_B,Lan,100,None,06:79:31:CB:84:75,ens7,ens7



Time to print interfaces 347 seconds


## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the TCP Traffic Notebook, you can ether open it on the explorer or click [Here](./TCP_Traffic.ipynb) to open the next notebook.